## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [6]:
reader = PdfReader("me/Profile.pdf") #read the pdf file
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

   
Contact
agjpikelis@gmail.com
www.linkedin.com/in/alexander-
pikelis-127042202 (LinkedIn)
Top Skills
AI Assistants
WatsonX
Inventory Optimization
Certifications
Data Science Tools
AI Data Scientist Assessment
UK Driving Licence
Machine Learning with Python
WatsonX: Designing & Building
Advanced AI Assistants
Alexander Pikelis
IBM Business Transformation Consultant | Charted Supply Chain AI
Professional | BSc (Hons) Business Analytics
London, England, United Kingdom
Experience
IBM
1 year 5 months
Business Transformation Consultant
October 2024 - Present (1 year 5 months)
DevOps Engineer
November 2024 - November 2025 (1 year 1 month)
Loughborough Sport
3 years 8 months
Lacrosse Club Treasurer 
April 2023 - October 2024 (1 year 7 months)
Responsible for the clubs budget, investments, expenditure reports and
expense claims for the 2023/2024 season
Loughborough Lacrosse Performance Athlete
September 2021 - October 2024 (3 years 2 months)
Current 1st String Defender for the Mens 1st Team 

In [8]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Alexander Pikelis"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n" #markdown tages for easier headings
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

"You are acting as Alexander Pikelis. You are answering questions on Alexander Pikelis's website, particularly questions related to Alexander Pikelis's career, background, skills and experience. Your responsibility is to represent Alexander Pikelis for interactions on the website as faithfully as possible. You are given a summary of Alexander Pikelis's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Alexander Pikelis. I'm a Business Transformation Consultant at IBM UK and an aspiring data scientist. I was born and currently live in Buckinghamshire.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nagjpikelis@gmail.com\nwww.linkedin.com/in/alexander-\npikelis-127042202 (LinkedIn)\nTop Skills\nAI Assistants\nWatsonX\nInventory Optimization\nCertifications\nData Science Tools\nAI Data Scientist 

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [15]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [16]:
#evaulator optimiser design pattern
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [17]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [30]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

OpenAI = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [42]:
def evaluate(reply, message, history) -> Evaluation: #takes a reply, message and history and returns an evaluation(BaseModel)
    #LLM able to output as an object is through structure output
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]

    response = OpenAI.chat.completions.parse(model="gpt-4o", messages=messages, response_format=Evaluation) 
    #parse is used to parse the response into the Evaluation model (schema)
    return response.choices[0].message.parsed

In [39]:
#create a message to test the evaluator
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you have a degree?"}]
response = openai.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
reply = response.choices[0].message.content

In [40]:
reply

'{"is_acceptable":true,"feedback":"Yes, I hold a Bachelor of Science (BSc) with Honours in Business Analytics from Loughborough University, which I completed in 2024."}'

In [43]:
evaluate(reply, "do you have a degree?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable as it accurately reflects Alexander Pikelis's educational background, specifically mentioning the degree and the institution where it was obtained. It is clear, concise, and maintains a professional tone suitable for a potential client or employer interaction. Additionally, providing the year of completion adds to the clarity and relevance of the response.")

In [44]:
#create a rerun function to retry the answer if is_acceptable is false
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "degree" in message: #if statement used to test the evaluator - if 'degree' it will respond in pig latin and evaluator/rerun will fail it and correct
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [48]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The response is not acceptable because it appears to be written in Pig Latin, which is unprofessional and not appropriate for the intended audience. Additionally, the information provided is incorrect. According to the provided details, Alexander Pikelis holds a Bachelor of Science in Business Analytics, not a Master's degree. The agent should respond in a professional and clear manner, accurately reflecting the qualifications as presented in the context.
